In [ ]:
import pandas as pd
import numpy as np
df = pd.read_csv('train_indessa.csv')

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
pip install sweetviz

In [ ]:
df_test = pd.read_csv('test_indessa.csv')

In [ ]:
df_test.head()

In [ ]:
df_test.info()

In [ ]:
#importing sweetviz
#import sweetviz as sv
#analyzing the dataset
#report = sv.analyze(df)
#display the report
#report.show_html('train-bank.html')


In [ ]:
#from pandas_profiling import ProfileReport
#design_report = ProfileReport(df)
#design_report.to_file(output_file='report.html')

In [ ]:
#pip install pandas-profiling

In [ ]:
df['loan_status'].value_counts()

In [ ]:
df.columns

In [ ]:

# Replaces months string with '' 
df['term'].replace(to_replace=' months', value='', regex=True, inplace=True) 

# Convert it to numeric
df['term'] = pd.to_numeric(df['term'], errors='coerce')

errors='coerce'
It will ignore all non-numeric values. Output: Code #3: Using errors='coerce'. It will replace all non-numeric values with NaN

In [ ]:
df['term'].value_counts()

In [ ]:
df['emp_length'].replace('n/a', '0', inplace=True)
df['emp_length'].replace(to_replace='\+ years', value='', regex=True, inplace=True)
df['emp_length'].replace(to_replace=' years', value='', regex=True, inplace=True)
df['emp_length'].replace(to_replace='< 1 year', value='0', regex=True, inplace=True)
df['emp_length'].replace(to_replace=' year', value='', regex=True, inplace=True)

# Convert it to numeric
df['emp_length'] = pd.to_numeric(df['emp_length'], errors='coerce')

In [ ]:
df['sub_grade'].replace(to_replace='A', value='0', regex=True, inplace=True)
df['sub_grade'].replace(to_replace='B', value='1', regex=True, inplace=True)
df['sub_grade'].replace(to_replace='C', value='2', regex=True, inplace=True)
df['sub_grade'].replace(to_replace='D', value='3', regex=True, inplace=True)
df['sub_grade'].replace(to_replace='E', value='4', regex=True, inplace=True)
df['sub_grade'].replace(to_replace='F', value='5', regex=True, inplace=True)
df['sub_grade'].replace(to_replace='G', value='6', regex=True, inplace=True) 

# Convert it to numeric
df['sub_grade'] = pd.to_numeric(df['sub_grade'], errors='coerce')

In [ ]:
df['last_week_pay'].replace(to_replace='th week', value='', regex=True, inplace=True)

df['last_week_pay'].replace(to_replace='NA', value='', regex=True, inplace=True)

# Convert it to numeric
df['last_week_pay'] = pd.to_numeric(df['last_week_pay'], errors='coerce')

### Handling Missing Values

In [ ]:
df.isnull().sum()

In [ ]:

columns = ['term', 'loan_amnt', 'funded_amnt', 'last_week_pay', 'int_rate', 'sub_grade',
           'annual_inc', 'dti', 'mths_since_last_delinq', 'mths_since_last_record', 'open_acc',
           'revol_bal', 'revol_util', 'total_acc', 'total_rec_int', 'mths_since_last_major_derog', 
           'tot_coll_amt', 'tot_cur_bal', 'total_rev_hi_lim', 'emp_length']
for col in columns:
    df[col].fillna(df[col].median(), inplace=True)  # Filling NaN values with median of each column present in columns.

  
num_cols = ['acc_now_delinq', 'total_rec_late_fee', 'recoveries', 'collection_recovery_fee', 'collections_12_mths_ex_med']
for col in num_cols:
    df[col].fillna(0, inplace=True)        ## Filling NaN values with 0 for each column present in columns.

In [ ]:

# Converts categorical variable into dummy/indicator variables.
cate_attr = ['home_ownership', 'purpose']
for cat in cate_attr:
   df_col = [cat]
   df[cat] = df[cat].astype("category")
   df[cat] = pd.get_dummies(df, columns=df_col)

In [ ]:
final_df = df.copy()
final_df = final_df.drop(['loan_status'], axis=1)

In [ ]:
final_df['income_to_loan'] = np.round_(final_df['annual_inc']/final_df['funded_amnt'], decimals = 2)

# How big the loan a person has taken with respect to his earnings, annual income to fund by investors ratio
final_df['income_to_loan_inv'] = np.round_(final_df['annual_inc']/final_df['funded_amnt_inv'], decimals = 2)


# Interest paid so far = interest received till date + Late fee received till date
final_df['total_int_paid'] = final_df['total_rec_int'] + final_df['total_rec_late_fee']

# Calculating EMIs (monthly)
#EMI = [P x R x (1+R)^N] / [(1+R)^N-1]
principal = final_df['loan_amnt']
rate = final_df['int_rate'] /(100*12)
time = final_df['term']

final_df['emi_per_month'] = np.round_((principal * rate * (1+rate)**time)/((1+rate)**(time)-1), decimals = 2)


# Calculating EMIs paid (in terms of percent) np.round_(in_array, decimals = 2)
final_df['emi_paid_perc'] = np.round_(((final_df['last_week_pay'])/(final_df['term']/12*52+1))*100, decimals = 2)

In [ ]:
cols = ['grade', 'emp_title', 'verification_status_joint', 'verification_status',
        'pymnt_plan', 'desc', 'purpose', 'title', 'zip_code', 'addr_state', 'initial_list_status', 'application_type']

for col in cols:
    final_df[col].fillna('missing', inplace=True)
    
    
final_df.fillna(0)  
final_df.reset_index(inplace=True)  # Resets the indices.

In [ ]:
final_df.drop(['desc', 'zip_code', 'emp_title', 'title', 'batch_enrolled','grade'], axis=1, inplace=True)

In [ ]:
# Class Label : Binary Classification
train_target = pd.DataFrame(df['loan_status'])

In [ ]:

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(np.array(final_df), np.array(train_target), test_size=0.30)

In [ ]:
final_df.head()

In [ ]:

import matplotlib.pyplot as plt
import sklearn.metrics as metrics
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from sklearn.metrics import roc_curve, auc 
from sklearn.metrics import roc_auc_score

In [ ]:
#CATBoost
from catboost import CatBoostClassifier
cat_clf = CatBoostClassifier(learning_rate=0.1, n_estimators=1400, subsample=0.70, max_depth=6, scale_pos_weight=2.5, silent=True)
cat_clf.fit(X_train, y_train)

# evaluate predictions
y_train_predict_cat = cat_clf.predict(X_train)
print("==========================")
print('Train Accuracy: %.3f' % metrics.accuracy_score(y_train, y_train_predict_cat))

# make predictions for test data
y_pred_cat = cat_clf.predict(X_test)
predictions = [round(value) for value in y_pred_cat]


accuracy_per_roc_auc = roc_auc_score(y_test, predictions)
print("ROC-AUC: %.10f%%" % (accuracy_per_roc_auc * 100))

print('Test Accuracy: %.3f' % metrics.accuracy_score(y_test, predictions))
print("==========================")
print(metrics.confusion_matrix(y_test, predictions))
print("==========================")
print(metrics.classification_report(y_test, predictions))
print("==========================")
print('Precision Score: %.3f' % metrics.precision_score(y_test, predictions))
print('Recall Score: %.3f' % metrics.recall_score(y_test, predictions))
print('F1 Score: %.3f' % metrics.f1_score(y_test, predictions))

In [ ]:

y_train_pred = cat_clf.predict(X_train)    
y_test_pred = cat_clf.predict(X_test)


train_fpr, train_tpr, train_thresholds = roc_curve(y_train, y_train_pred)
test_fpr, test_tpr, test_thresholds = roc_curve(y_test, y_test_pred)

plt.plot(train_fpr, train_tpr, label="Train AUC ="+str(auc(train_fpr, train_tpr)))
plt.plot(test_fpr, test_tpr, label="Test AUC ="+str(auc(test_fpr, test_tpr)))
plt.legend()
plt.xlabel("True Positive Rate(TPR)", fontweight='bold')
plt.ylabel("False Positive Rate(FPR)", fontweight='bold')
plt.title("ROC-AUC", fontweight='bold')
plt.grid()
plt.show()

In [ ]:
#XGBOOST
from xgboost import XGBClassifier

xg = XGBClassifier(scale_pos_weight=2.5 ,silent=True)
    
xg.fit(X_train, y_train)


# evaluate predictions
y_train_predict_xg = xg.predict(X_train)
print("==========================")
print('Train Accuracy %.3f' % metrics.accuracy_score(y_train, y_train_predict_xg))

# make predictions for test data
y_pred_xg = xg.predict(X_test)
predictions = [round(value) for value in y_pred_xg]

accuracy_per_roc_auc = roc_auc_score(y_test, predictions)
print("ROC-AUC: %.10f%%" % (accuracy_per_roc_auc * 100))


print('Test Accuracy %.3f' % metrics.accuracy_score(y_test, predictions))
print("==========================")
print(metrics.confusion_matrix(y_test, predictions))
print("==========================")
print(metrics.classification_report(y_test, predictions))
print("==========================")
print('Precision Score %.3f' % metrics.precision_score(y_test, predictions))
print('Recall Score %.3f' % metrics.recall_score(y_test, predictions))
print('F1 Score %.3f' % metrics.f1_score(y_test, predictions))

In [ ]:

y_train_pred = xg.predict(X_train)    
y_test_pred = xg.predict(X_test)


train_fpr, train_tpr, tr_thresholds = roc_curve(y_train, y_train_pred)
test_fpr, test_tpr, te_thresholds = roc_curve(y_test, y_test_pred)

plt.plot(train_fpr, train_tpr, label="Train AUC ="+str(auc(train_fpr, train_tpr)))
plt.plot(test_fpr, test_tpr, label="Test AUC ="+str(auc(test_fpr, test_tpr)))
plt.legend()
plt.xlabel("True Positive Rate(TPR)", fontweight='bold')
plt.ylabel("False Positive Rate(FPR)", fontweight='bold')
plt.title("ROC-AUC", fontweight='bold')
plt.grid()
plt.show()